In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('adult/adult.data', header=None, index_col=False,
                   names=['年龄','单位性质','权重','学历','受教育时长','婚姻状况','职业','家庭状况','种族','性别',
                          '资产所得','资产损失','周工作时长','原籍','收入'])

data_dummies = pd.get_dummies(data)

features = data_dummies.loc[:,'年龄':'原籍_ Yugoslavia']

X = features.values
y = data_dummies['收入_ >50K'].values

In [4]:
print(X.shape)
print(y.shape)

(32561, 108)
(32561,)


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=38)

In [6]:
from sklearn import tree

dtc = tree.DecisionTreeClassifier()
dtc.fit(X_train,y_train)
print(dtc.score(X_train,y_train))
print(dtc.score(X_test,y_test))

0.9999590499590499
0.8127994103918438


In [7]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()
clf.fit(X_train,y_train)
print(clf.score(X_train,y_train))
print(clf.score(X_test,y_test))

0.833947583947584
0.7739835401056381
